In [45]:
import sqlite3
import pandas as pd

# Path to the SQLite database file
db_file = "toms_code/data/surgery_data.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

#get data into pandas DFs
query = "SELECT * FROM specialties" 
specialties = pd.read_sql_query(query, conn)
query = "SELECT * FROM procedures" 
procedures = pd.read_sql_query(query, conn)
query = "SELECT * FROM procedure_association"  
procedure_association = pd.read_sql_query(query, conn)
query = "SELECT * FROM surgeries" 
surgeries = pd.read_sql_query(query, conn)
query = "SELECT * FROM sessions"
sessions = pd.read_sql_query(query, conn)

In [46]:
specialties.head(5)

,id,name
0,0,General Surgery
1,1,Orthopaedic Services
2,2,Gynaecology Services
3,3,"Ear, Nose and Throat Surgical Services"
4,4,Urology Services


In [47]:
procedures.head(5)

,id,description,code
0,1,bowel resection hartmanns procedure,16564004.0
1,2,laparotomy,86481000.0
2,3,peritoneal lavage intra abdominal washout,NaN
3,4,excisional debridement wound infection or burn,473218004.0
4,5,orif tibia and fibula,702489005.0


In [48]:
procedure_association.head(5)

,surgeries_id,procedures_id
0,1,2
1,60,2
2,79,2
3,98,2
4,131,2


In [49]:
#filter surgeries so planned = 1
surgeries = surgeries[surgeries['planned']==1]
#engineer surgery_overtime
def calculate_overtime(row):
    return max(0, row['total_duration'] - row['scheduled_duration'])
surgeries['surgery_overtime'] = surgeries.apply(lambda row: calculate_overtime(row),axis=1)

surgeries.head(5)

,id,facility,specialty_id,surgical_consultant_id,admission_type,planned,anaesthesia_type,asa_rating,primary_procedure_id,arrival_datetime,due_date_datetime,scheduled_duration,total_duration,turn_around_time,predicted_duration,predicted_variance,complete_date_datetime,surgery_overtime
14,15,A,2,4,Waiting List,1,None,3.0,24,2015-01-23 06:49:00.000000,2015-05-03 06:49:00.000000,232.0,122.0,42.0,102.241065,1186.336753,2015-02-02 08:00:00,0.0
15,16,A,1,5,Waiting List,1,None,3.0,26,2015-01-13 06:52:00.000000,2015-04-23 06:52:00.000000,240.0,84.0,25.0,104.770368,1245.759452,2015-02-02 08:00:00,0.0
16,17,A,1,6,Waiting List,1,None,2.0,26,2014-11-02 06:55:00.000000,2015-02-10 06:55:00.000000,240.0,124.0,17.0,123.569556,1732.926825,2015-02-02 08:00:00,0.0
17,18,A,1,6,Waiting List,1,None,1.0,27,2014-12-19 07:00:00.000000,2015-03-29 07:00:00.000000,60.0,33.0,16.0,57.307570,372.718744,2015-02-02 08:00:00,0.0
19,20,A,2,4,Waiting List,1,None,1.0,28,2015-01-20 07:02:00.000000,2015-04-30 07:02:00.000000,404.0,69.0,23.0,111.631941,1414.276105,2015-02-02 08:00:00,0.0


In [50]:
#filter surgeries so planned = 1
sessions = sessions[sessions['planned']==1]
sessions.head()

,id,specialty_id,facility,planned,surgical_consultant_id,start_time,duration,theatre_number
0,1,1,A,1,36,2016-03-01 08:00:00.000000,540.0,1.0
1,2,1,A,1,33,2015-04-01 08:00:00.000000,540.0,1.0
2,3,1,A,1,59,2016-04-01 08:00:00.000000,540.0,1.0
3,4,1,A,1,59,2016-06-01 08:00:00.000000,540.0,1.0
4,5,1,A,1,59,2015-07-01 08:00:00.000000,540.0,1.0


In [51]:
import sweetviz as sv

# Analyze the DataFrame
report_surgeries = sv.analyze(surgeries)
report_sessions = sv.analyze(sessions)

                                             |          | [  0%]   00:00 -> (? left)

                                             |          | [  0%]   00:00 -> (? left)

In [52]:
# Display the sessions exploratory analysis
# report_sessions.show_html("sweetviz_report_sessions.html")

In [53]:
# Display the surgeries exploratory analysis
# report_surgeries.show_html("sweetviz_report_surgeries.html")

## Questions
- What are the different admission types A: waiting list, arranged admissions
- What are the different asa ratings? A: 1-4
- How many surgeries run overtime? A: Just under 30%
- Do planned and emergency surgeries share surgery session resources?
- What are the different facilities? A: A,B,C
- What's the difference between arrival and due date datetimes?
- What is arranged admissions?
- What's predicted duration?
- Is turnaround_time considered in shceduled_duration?

What is the earliest and latest start times for each session and specialty?

In [54]:
# Convert 'start_time' to datetime
sessions['start_time'] = pd.to_datetime(sessions['start_time'])

# Group by 'facility' and 'specialty_id' and find earliest and latest 'start_time'
earliest_latest_times = sessions.groupby(['facility', 'specialty_id'])['start_time'].agg(['min', 'max']).reset_index()

print(earliest_latest_times)

   facility  specialty_id                 min                 max
0         A             0 2015-02-02 08:00:00 2017-01-31 08:00:00
1         A             1 2015-02-02 08:00:00 2017-01-31 08:00:00
2         A             2 2015-02-02 08:00:00 2017-01-25 08:00:00
3         A             3 2015-07-31 08:00:00 2017-01-27 08:00:00
4         A             4 2015-02-04 08:00:00 2017-01-26 08:00:00
5         A             5 2015-02-10 08:00:00 2017-01-24 08:00:00
6         B             0 2015-02-02 08:30:00 2017-01-31 13:30:00
7         B             1 2015-02-02 08:30:00 2017-01-31 08:30:00
8         B             2 2015-02-02 08:30:00 2017-01-31 13:30:00
9         B             3 2015-02-02 08:30:00 2017-01-31 08:30:00
10        B             4 2015-02-03 08:30:00 2017-01-31 08:30:00
11        B             5 2015-02-13 08:30:00 2015-09-25 09:00:00
12        B             9 2016-08-10 08:30:00 2016-08-10 08:30:00
13        B            10 2016-08-26 09:00:00 2016-08-26 09:00:00
14        

In [55]:
import pandas as pd

# Grouping by 'specialty_id' and 'facility', then counting the entries
counts = sessions.groupby(['specialty_id', 'facility']).size().reset_index(name='count')

print(counts)


    specialty_id facility  count
0              0        A    709
1              0        B   1058
2              0        C    283
3              1        A    661
4              1        B    781
5              1        C     38
6              2        A    276
7              2        B    643
8              2        C    125
9              3        A     45
10             3        B    801
11             3        C    218
12             4        A    162
13             4        B    317
14             4        C     95
15             5        A     41
16             5        B     19
17             6        C     23
18             9        B      1
19            10        B      1


Finding reasonable specialty and facility for testing (want a small number of surgeries)

In [57]:
print(len(sessions))

# Convert 'start_time' column to datetime type if it's not already
sessions['start_time'] = pd.to_datetime(sessions['start_time'])

print(len(sessions))

# Filter rows where start_time is '2016-04-01'
filtered_sessions = sessions[(sessions['start_time'].dt.date < pd.to_datetime('2015-12-01').date()) & (sessions['start_time'].dt.date > pd.to_datetime('2015-11-01').date())]

# Group by 'specialty_id' and 'facility', then count the number of rows
surgeries_count = filtered_sessions.groupby(['specialty_id', 'facility']).size().reset_index(name='surgeries_count')

print(surgeries_count)


6297
6297
    specialty_id facility  surgeries_count
0              0        A               29
1              0        B               50
2              0        C               10
3              1        A               24
4              1        B               36
5              1        C                1
6              2        A               14
7              2        B               27
8              2        C                4
9              3        A                3
10             3        B               33
11             3        C               13
12             4        A                8
13             4        B               13
14             4        C                5
15             5        A                2
16             6        C                1
